# Sparse Hebbian Learning: plotting tools

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(precision=4, suppress=True)

In [3]:
from shl_scripts.shl_experiments import SHL

DEBUG_DOWNSCALE, verbose = 1, 0

In [4]:
help(SHL)

Help on class SHL in module shl_scripts.shl_experiments:

class SHL(builtins.object)
 |  Base class to define SHL experiments:
 |      - initialization
 |      - coding and learning
 |      - visualization
 |      - quantitative analysis
 |  
 |  Methods defined here:
 |  
 |  __init__(self, height=256, width=256, patch_size=(16, 16), datapath='database/', n_dictionary=324, learning_algorithm='mp', fit_tol=None, l0_sparseness=15, n_iter=16384, eta=0.005, eta_homeo=0.01, nb_quant=128, C=0.0, do_sym=False, alpha_homeo=0.0, max_patches=4096, batch_size=512, record_each=128, n_image=200, DEBUG_DOWNSCALE=1, verbose=0, data_cache='/home/laurent/tmp/data_cache')
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  code(self, data, dico, coding_algorithm='mp', matname=None, l0_sparseness=None, **kwargs)
 |  
 |  decode(self, sparse_code, dico)
 |  
 |  get_data(self, name_database='serre07_distractors', seed=None, patch_norm=True, matname=None, **kwargs)
 |  
 |  lea

In [5]:
matname = 'vanilla'
shl = SHL(DEBUG_DOWNSCALE=DEBUG_DOWNSCALE, learning_algorithm='mp', verbose=verbose)
data = shl.get_data(name_database='serre07_distractors', matname=matname)

FileNotFoundError: [Errno 2] No such file or directory: '/home/laurent/tmp/data_cache/vanilla_data_lock'

In [ ]:
list_figures = ['show_dico',  'time_plot_prob',  'time_plot_kurt',  'time_plot_var']
dico = shl.learn_dico(matname=matname, list_figures=list_figures)

In [ ]:
sparse_code = shl.code(data, dico, matname=matname)
shl.plot_variance(sparse_code, data=data);

In [ ]:
shl.plot_variance_histogram(sparse_code, data=data);

## Version used

In [ ]:
%load_ext version_information
%version_information numpy, shl_scripts